# Virtual Concierge 

## Face Recognition Project with MXNet

***
Copyright [2017]-[2018] Amazon.com, Inc. or its affiliates. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. A copy of the License is located at

http://aws.amazon.com/apache2.0/

or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
***

### Prerequisites:

#### Python package dependencies

The following packages need to be installed before proceeding:

* Boto3 - `pip install boto3`
* MXNet - `pip install mxnet`
* numpy - `pip install numpy`
* OpenCV - `pip install opencv-python`
* Graphviz - `pip install graphviz`
* Matplotlib - `pip install matplotlib`
* Seaborn - `pip install seaborn`

### Import dependencies

Verify that all dependencies are installed using the cell below. Continue if no errors encountered, warnings can be ignored.

In [ ]:
from __future__ import print_function

import boto3
import cv2
import sys
import numpy as np
import mxnet as mx
import os
import json
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sns 

%matplotlib inline

### Load pretrained model

`get_model()` : Loads MXNet symbols and params, defines model using symbol file and binds parameters to the model using params file.

In [ ]:
def get_model(ctx, image_size, model_str, layer):
    _vec = model_str.split(',')
    assert len(_vec)==2
    prefix = _vec[0]
    epoch = int(_vec[1])
    print('loading',prefix, epoch)
    sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)
    all_layers = sym.get_internals()
    sym = all_layers[layer+'_output']
    model = mx.mod.Module(symbol=sym, context=ctx, label_names = None)
    model.bind(data_shapes=[('data', (1, 3, image_size[0], image_size[1]))])
    model.set_params(arg_params, aux_params)
    return model, sym

### Preprocess images

In order to input only face pixels into the network, all input images are passed through a pretrained face detection and alignment model as described above. The output of this model are landmark points and a bounding box corresponding to the face in the image. Using this output, the image is processed using affine transforms to generate the aligned face images which are input to the network. The functions performing this is defined below.

`get_input()` : Returns aligned face to the bbox and margin, and [rotation](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html)

`show_input()` : Shows the image after transposing it

In [ ]:
def get_input(img, image_size, bbox=None, margin=0, rotate=0):
    if bbox is None:
        det = np.zeros(4, dtype=np.int32)
        det[0] = int(img.shape[1]*0.0625)
        det[1] = int(img.shape[0]*0.0625)
        det[2] = img.shape[1] - det[0]
        det[3] = img.shape[0] - det[1]
    else:
        det = bbox
    # Crop
    bb = np.zeros(4, dtype=np.int32)
    bb[0] = np.maximum(det[0]-margin/2, 0)
    bb[1] = np.maximum(det[1]-margin/2, 0)
    bb[2] = np.minimum(det[2]+margin/2, img.shape[1])
    bb[3] = np.minimum(det[3]+margin/2, img.shape[0])
    img = img[bb[1]:bb[3],bb[0]:bb[2],:]
    # Rotate if required
    if 0 < rotate and rotate < 360:
        rows,cols,_ = img.shape
        M = cv2.getRotationMatrix2D((cols/2,rows/2),360-rotate,1)
        img = cv2.warpAffine(img,M,(cols,rows))
    # Resize and transform
    img = cv2.resize(img, (image_size[1], image_size[0]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    aligned = np.transpose(img, (2,0,1))
    return aligned

def show_input(aligned):
    plt.imshow(np.transpose(aligned,(1,2,0)))

### Get Features

`l2_normalize()`: Performs row normalization on the vector

`get_feature()` : Performs forward pass on the data aligned using model and returns the embedding

In [ ]:
def l2_normalize(X):
    norms = np.sqrt((X * X).sum(axis=1))
    X /= norms[:, np.newaxis]
    return X

def get_feature(model, aligned):
    input_blob = np.expand_dims(aligned, axis=0)
    data = mx.nd.array(input_blob)
    db = mx.io.DataBatch(data=(data,))
    model.forward(db, is_train=False)
    embedding = model.get_outputs()[0].asnumpy()
    embedding = l2_normalize(embedding).flatten()
    return embedding

### Visualize Model

Load the pre-trained mobilenet mobile, setting the context to cpu and visualize the architecture.

In [ ]:
%%time

image_size = (112,112)
model_name = './models/mobilenet1,0'
model, sym = get_model(mx.cpu(), image_size, model_name, 'fc1')

In [ ]:
mx.viz.plot_network(sym)

### Evaulate

Download sample image, and extract face coordinates

In [ ]:
!aws s3 cp s3://aiml-lab-sagemaker/politicians/politicians2.jpg tmp/image

In [ ]:
rekognition = boto3.client('rekognition')

def get_bboxes(img, margin=0):
    # Detect faces
    ret, buf = cv2.imencode('.jpg', img)
    ret = rekognition.detect_faces(
        Image={
            'Bytes': buf.tobytes()
        },
        Attributes=['DEFAULT'],
    )
    # Return the bounding boxes for each face
    height, width, _ = img.shape
    bboxes = []
    for face in ret['FaceDetails']:
        box = face['BoundingBox']
        x1 = int(box['Left'] * width)
        y1 = int(box['Top'] * height)
        x2 = int(box['Left'] * width + box['Width'] * width)
        y2 = int(box['Top'] * height + box['Height']  * height)
        bboxes.append((x1, y1, x2, y2))
    return bboxes

For each of the coordinates, get a the aligned image, and draw the rectangle

In [ ]:
%%time

# Load the image, and get bboxes
img = cv2.imread('tmp/image')
boxes = get_bboxes(img)

# blue, green, red, grey
colors = ((220,220,220),(242,168,73),(76,182,252),(52,194,123))

img_aligned = []
for col, bbox in enumerate(boxes): 
    aligned = get_input(img, image_size, bbox)
    img_aligned.append(aligned)
    cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), colors[col%4], 3)
    
# Plot the figure in it's original rotation
plt.figure(figsize=(10,10))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
# output the aligned image
fig = plt.figure(figsize=(10,10))
for i, aligned in enumerate(img_aligned):
    a = fig.add_subplot(1, len(img_aligned), i+1)
    a.set_title('Image {}'.format(i))
    show_input(aligned)
plt.show()

### Generate embedding

Pass each face through the network sequentially to generate embedding vectors for each. 

In [ ]:
img_vecs = np.array([get_feature(model, aligned) for aligned in img_aligned])
print(img_vecs.shape)
img_vecs[0]

### Calculate similarity

Calculate the cosine similarity between the embedding vectors to see how similar they are to each out. 

Similarity values in [-1,1].

In [ ]:
sims = np.dot(img_vecs, img_vecs.T)
np.fill_diagonal(sims, 0)
sns.heatmap(sims, annot=True, fmt=".03f")

### Vectorize Dataset

Download a the politician dataset, and vectories the images.

In [ ]:
!mkdir -p tmp/images
!aws s3 sync s3://aiml-lab-sagemaker/actors/ tmp/images

In [ ]:
%%time

image_dir = 'tmp/images'
names = []
vecs = []

for file in os.listdir(image_dir):
    name = file.split('.')[0]
    img = cv2.imread(os.path.join(image_dir, file))
    bboxes = get_bboxes(img)
    bbox = bboxes[0]
    print(name, bbox)
    aligned = get_input(img, image_size, bbox)
    vec = get_feature(model, aligned)   
    names.append(name)
    vecs.append(vec)
    
vecs = np.array(vecs)

Save the vectors back to a file with the names.

In [ ]:
np.savez('people.npz', names=names, vecs=vecs)

### Plot Distribution

Compare the vectors of all the politications to input image, plot the distribution and outliner for match.

In [ ]:
img = img_vecs[0]

# calculate cosine similarity and relative zscores
sims = np.dot(vecs, img)
zscores = stats.zscore(sims)

# plot series and print score and name
sns.set(color_codes=True)
plt.figure(figsize=(10,6))
ax = sns.distplot(zscores, bins=50, kde=False, rug=True)
ax.set(xlabel='zscore', ylabel='number of people')
plt.title('zscore distribution')
plt.show()

Output the name of the highest similarity based on the dataset

In [ ]:
from math import erf, sqrt
def phi(x):
    #'Cumulative distribution function for the standard normal distribution'
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

idx = sims.argmax()
print('sim: {}, zscore: {}, prob: {}, name: {}'.format(sims[idx], zscores[idx], phi(zscores[idx]), names[idx]))

## Set up hosting for the model

### Export the model from mxnet

In order to set up hosting, we have to [import the model from training to hosting](https://aws.amazon.com/blogs/machine-learning/bring-your-own-pre-trained-mxnet-or-tensorflow-models-into-amazon-sagemaker/). We will begin by exporting the model from MXNet and saving it down. The exported model has to be converted into a form that is readable by ``sagemaker.mxnet.model.MXNetModel``. The following code describes exporting the model in a form that does the same:

In [ ]:
import os
import json

os.mkdir('model')

model.save_checkpoint('model/model', 0000)
with open ('model/model-shapes.json', "w") as shapes:
    json.dump([{"shape": model.data_shapes[0][1], "name": "data"}], shapes)

import tarfile

def flatten(tarinfo):
    tarinfo.name = os.path.basename(tarinfo.name)
    return tarinfo
    
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model", filter=flatten)
tar.close()

The above piece of code essentially hacks the MXNet model export into a sagemaker-readable model export. Study the exported model files if you want to organize your exports in the same fashion as well. Alternatively, you can load the model on MXNet itself and load the sagemaker model as you normally would. Refer [here](https://github.com/aws/sagemaker-python-sdk#model-loading) for details on how to load MXNet models.

### Import model into SageMaker

Open a new sagemaker session and upload the model on to the default S3 bucket. We can use the ``sagemaker.Session.upload_data`` method to do this. We need the location of where we exported the model from MXNet and where in our default bucket we want to store the model(``/model``). The default S3 bucket can be found using the ``sagemaker.Session.default_bucket`` method.

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()
role

In [ ]:
model_data = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='virtual-concierge')

Use the ``sagemaker.mxnet.model.MXNetModel`` to import the model into SageMaker that can be deployed. We need the location of the S3 bucket where we have the model, the role for authentication and the entry_point where the model defintion is stored (``predict.py``). 

In [ ]:
from sagemaker.mxnet.model import MXNetModel
sagemaker_model = MXNetModel(model_data = model_data, role = role, entry_point = 'predict.py')

### Create endpoint

Now the model is ready to be deployed at a SageMaker endpoint. We can use the ``sagemaker.mxnet.model.MXNetModel.deploy`` method to do this. Unless you have created or prefer other instances, we recommend using 1 ``'ml.c4.xlarge'`` instance for this training. These are supplied as arguments. 

In [ ]:
import logging
logging.getLogger().setLevel(logging.WARNING)

predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Making an inference request

Now that our Endpoint is deployed and we have a ``predictor`` object which we can call for inference.

We wiill pass a single batch of an aligned image as a numpy array in the shape the model expects, setting `content_type` and `serializer` to convert into bytes.  The `predict.py` endpoint includes overides for `model_fn` to load fully connected layer and `transform_fn` to [transform](https://sagemaker.readthedocs.io/en/stable/using_mxnet.html?highlight=input_fn#using-input-fn-predict-fn-and-output-fn) to load numpy input and return normalized emeddings as json.

The SageMaker MXNet containers are [open source](https://github.com/aws/sagemaker-containers) if you needed more details.

In [ ]:
!cat predict.py

In [ ]:
# Pass last aligned input into model
input = np.expand_dims(aligned, axis=0)
input.shape

In [ ]:
def numpy_bytes_serializer(data):
    import io
    import numpy as np
    
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

In [ ]:
from predict import transform_fn

data = numpy_bytes_serializer(input)
embedding, content_type = transform_fn(model, data, 'application/x-npy', 'application/json')
np.array(json.loads(embedding))[:10]

In [ ]:
%%time

# Set the content-type to numpy 
predictor.serializer = numpy_bytes_serializer
predictor.accept = 'application/json'
predictor.content_type = 'application/x-npy'
response = predictor.predict(input)
print(np.array(response)[:10])

### Opimtize your model with Neo API

Neo API allows to optimize our model for a specific hardware type. When calling `compile_model()` function, we specify the target instance family (C5) as well as the S3 bucket to which the compiled model would be stored.

*Important*. If the following command result in a permission error, scroll up and locate the value of execution role returned by `get_execution_role()`. The role must have access to the S3 bucket specified in ``output_path``.**

In [ ]:
input_shape = model.data_shapes[0][1]
output_path = '/'.join(model_data.split('/')[:-1])
target_instance_family = 'ml_m4'  # 'rasp3b'
compiled_model = sagemaker_model.compile(job_name=sagemaker_model.name+'-compile',
                                   target_instance_family=target_instance_family, 
                                   input_shape={'data':input_shape},
                                   role=role,
                                   output_path=output_path)

### Creating an inference Endpoint

We can deploy this compiled model, note that we need to use the same instance that the target we used for compilation. This creates a SageMaker endpoint that we can use to perform inference. 

The arguments to the ``deploy`` function allow us to set the number and type of instances that will be used for the Endpoint. Make sure to choose an instance for which you have compiled your model, so in our case  `ml_c5`. Neo API uses a special runtime (DLR runtime), in which our optimzed model will run.

In [ ]:
compiled_predictor = compiled_model.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

This endpoint will receive uncompressed NumPy arrays, whose Content-Type is given as `application/x-npy`:

### Making an inference request

Now that our Endpoint is deployed and we have a ``compiled_predictor`` object which we can call.

In [ ]:
%%time

# Get the compiled predictor response
compiled_predictor.accept = 'application/json'
compiled_predictor.content_type = 'application/x-npy'
compiled_predictor.serializer = numpy_bytes_serializer
response = compiled_predictor.predict(input)
print(np.array(response)[:10])

In [ ]:
%%time 

# Compare this with the inline model
local_embedding = get_feature(model, aligned)
print(local_embedding[:10])

## Setting up Lambda Layers

### Create Model layer

Copy the predict.py into `python` folder with the untared compiled model, and create a new zip file for the lambda layer

In [ ]:
!rm -Rf python
!mkdir -p python/compiled
!aws s3 cp $compiled_model.model_data ./python

* Unzip the model and move to a `compiled` folder under the python directory.  
* Rename the compiled model, and remove the shape file that isn't required.
* Then copy the `predict.py` file and create a zip file which can be used as a layer

In [ ]:
%%bash

tar zxvf python/*.tar.gz -C python/compiled
mv python/compiled/compiled.params python/compiled/model.params
mv python/compiled/compiled_model.json python/compiled/model.json
mv python/compiled/compiled.so python/compiled/model.so
rm python/*.tar.gz python/compiled/model-shapes.json
cp predict.py ./python

In [ ]:
# Inspect the python library
!du -h python

### Publish Layer 

You can download the [predict_layer.zip](./predict_layer.zip).  Create a layers for predict.

In [ ]:
!zip -r predict_layer.zip ./python -x '*__pycache__*'
!aws lambda publish-layer-version --layer-name 'predict' \
    --compatible-runtimes python3.6 python3.7 \
    --zip-file fileb://predict_layer.zip

### Test Local lambda Layer

Download the DLR layer alongside the compiled model so that we can test inference.  

This and the Pillow layer are also available via ARN:

* [dlr](https://s3.amazonaws.com/deeplens-virtual-concierge-model/dlr-1.0-py3.7.zip) at ARN `arn:aws:lambda:us-east-1:882607831196:layer:dlr:4`
* [pillow](https://s3.amazonaws.com/deeplens-virtual-concierge-model/pillow-5.4.1-py3.7.zip) at ARN: `arn:aws:lambda:us-east-1:882607831196:layer:pillow:1`

In [ ]:
# Get the DLR runtime and unzip to the python directory for testing
!wget -q -nc https://s3.amazonaws.com/deeplens-virtual-concierge-model/dlr-1.0-py3.7.zip
!unzip -q -o dlr-1.0-py3.7.zip -x '*__pycache__*'

In [ ]:
# Add the relative python path for local testing
import sys
sys.path.append('./python')

In [ ]:
import predict
import json
import time

dlr_model = None

# Run the inference
if dlr_model == None:
    print('loading model')
    dlr_model = predict.neo_load(model_path='python/compiled')
    
output = predict.neo_inference(dlr_model, input)
predict.neo_postprocess(output)

### Create Lambda

TODO: Use SAM cli to create a lambda function with layers and invoke payload

In [ ]:
!cat handler.py

Get an image, and the bounding box and output an event payload to send to lambda

In [ ]:
# TODO: Load image bytes payload
import io
import base64
import boto3

rekognition = boto3.client('rekognition')

# Read an image
with open('tmp/image', 'rb') as f:
    payload = f.read()

# Call rekognition to get bbox
ret = rekognition.detect_faces(
    Image={
        'Bytes': payload
    },
    Attributes=['DEFAULT'],
)

# Create the lambda event
event = { 
    'Image': { 'Bytes': str(base64.b64encode(payload), 'utf-8') },
    'BoundingBox': ret['FaceDetails'][0]['BoundingBox'],
}

# Decode the payload
payload = base64.b64decode(event['Image']['Bytes'])
bbox = event['BoundingBox']
image = predict.neo_preprocess(payload, 'application/x-image', bbox=bbox)
image.shape

### Clean up

Delete the regular and neo endpoints

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

In [ ]:
sagemaker.Session().delete_endpoint(compiled_predictor.endpoint)

Clear all stored model data so that we don't overwrite them the next time. 

In [ ]:
import shutil
shutil.rmtree('tmp')
shutil.rmtree('model')
shutil.rmtree('python')
!rm *.zip *.tar.gz